# Dynamic Prompts & Long-Term Memory for HR Agents - LangChain 1.0

**Module:** Context Engineering - Dynamic Prompts & Persistent Memory

**Based on:** https://docs.langchain.com/oss/python/langchain/context-engineering

**What you'll learn:**
- 🎯 **Dynamic Prompts** with `@dynamic_prompt` middleware
- 📚 **Read** long-term memory in tools using `get_store()`
- ✍️ **Write** long-term memory in tools
- 💾 **Store** user preferences across sessions
- 🔍 **Query** stored memories with namespaces
- 🎨 **Combine** both for personalized experiences

**HR Use Cases:**
- Personalized greetings based on stored preferences
- Remember employee communication styles
- Store and retrieve work schedule preferences
- Track career goals and development plans

**Time:** 2-3 hours

---

## Setup: Install Dependencies

In [ ]:
!pip install --pre -U langchain langchain-openai langgraph
!pip install langgraph-checkpoint-sqlite

## Setup: Configure API Key

In [ ]:
from google.colab import userdata
import os

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

print("✅ API Key configured!")

## Import Libraries

In [ ]:
from typing import Annotated
from dataclasses import dataclass

# Core LangChain
from langchain.agents import create_agent
from langchain.agents.middleware import dynamic_prompt, ModelRequest
from langchain_core.tools import tool
from langchain_core.runnables import RunnableConfig

# LangGraph
from langgraph.prebuilt import InjectedStore
from langgraph.store.memory import InMemoryStore

# Utilities
from datetime import datetime
import json

print("✅ Imports successful!")

## Setup: Employee Database

In [ ]:
# Basic employee database
EMPLOYEE_DB = {
    "EMP101": {
        "name": "Priya Sharma",
        "department": "Engineering",
        "role": "Senior Developer",
        "leave_balance": 12
    },
    "EMP102": {
        "name": "Rahul Verma",
        "department": "Engineering",
        "role": "Engineering Manager",
        "leave_balance": 8
    },
    "EMP103": {
        "name": "Anjali Patel",
        "department": "HR",
        "role": "HR Director",
        "leave_balance": 15
    }
}

print(f"✅ Loaded {len(EMPLOYEE_DB)} employees")

---
# Part 1: Dynamic Prompts with @dynamic_prompt Middleware

**What is @dynamic_prompt?**
- Generates system prompts dynamically for each request
- Accesses runtime context, session state, and long-term memory
- Personalizes the agent based on user preferences

**Key Components:**
- `request.runtime.context` - Runtime context (user ID, etc.)
- `request.state` - Current session state
- `request.runtime.store` - Access to long-term memory

---

## Step 1: Define Context Schema

In [ ]:
# Define context schema that will be passed at runtime
@dataclass
class HRContext:
    """Runtime context for HR agent."""
    employee_id: str

print("✅ Context schema defined: HRContext(employee_id)")

## Step 2: Create @dynamic_prompt Middleware

This function generates personalized prompts using:
1. **Runtime context** - Employee ID
2. **Long-term memory** - Stored preferences
3. **Session state** - Conversation length

In [ ]:
@dynamic_prompt
def personalized_hr_prompt(request: ModelRequest) -> str:
    """
    Generate personalized HR prompt based on:
    - Runtime context (employee_id)
    - Long-term memory (preferences)
    - Session state (message count)
    """
    # 1. Access runtime context
    employee_id = request.runtime.context.employee_id
    
    # 2. Access long-term memory store
    store = request.runtime.store
    
    # 3. Access session state
    message_count = len(request.state.get("messages", []))
    
    # Look up employee info from database
    employee = EMPLOYEE_DB.get(employee_id, {})
    
    if not employee:
        return "You are a helpful HR assistant."
    
    name = employee.get("name", "Employee")
    department = employee.get("department", "Unknown")
    role = employee.get("role", "Unknown")
    leave_balance = employee.get("leave_balance", 0)
    
    # Time-based greeting
    hour = datetime.now().hour
    if hour < 12:
        greeting = "Good morning"
    elif hour < 17:
        greeting = "Good afternoon"
    else:
        greeting = "Good evening"
    
    # Base prompt
    prompt = f"""{greeting}! You are an AI HR assistant helping {name} ({employee_id}).

**Employee Context:**
- Name: {name}
- Department: {department}
- Role: {role}
- Leave Balance: {leave_balance} days
"""
    
    # Try to get communication preferences from long-term memory
    try:
        namespace = ("employee_preferences", employee_id)
        comm_prefs = store.get(namespace, "communication_style")
        
        if comm_prefs:
            style = comm_prefs.value.get("style", "balanced")
            response_format = comm_prefs.value.get("response_format", "paragraphs")
            
            prompt += f"\n**Communication Preferences (from memory):**"
            prompt += f"\n- Style: {style}"
            prompt += f"\n- Format: {response_format}"
            
            # Add style-specific instructions
            if style == "concise":
                prompt += "\n\n**Instructions:** Provide brief, to-the-point responses. Use bullet points."
            elif style == "detailed":
                prompt += "\n\n**Instructions:** Provide comprehensive explanations with examples."
            elif style == "formal":
                prompt += "\n\n**Instructions:** Maintain professional and formal tone."
        else:
            prompt += "\n\n**Instructions:** Be helpful and professional."
    
    except Exception as e:
        prompt += "\n\n**Instructions:** Be helpful and professional."
    
    # Add conversation length context
    if message_count > 10:
        prompt += "\n\n*Note: This is a long conversation. Focus on recent context and be concise.*"
    
    return prompt

print("✅ Dynamic prompt middleware created!")
print("\nThis prompt will:")
print("  • Greet user based on time of day")
print("  • Include employee context")
print("  • Load communication preferences from memory")
print("  • Adapt based on conversation length")

---
# Part 2: Long-Term Memory - Read & Write

**What is Long-Term Memory?**
- Persistent storage that survives across sessions
- Stores user preferences, history, and learned information
- Organized using namespaces

**Key Patterns:**
- Use `InjectedStore` to access the store in tools
- Use `get_runtime()` to get current user context
- Organize with namespaces: `("category", "user_id")`

---

## Step 1: Initialize Memory Store with Sample Data

In [ ]:
# Create in-memory store (in production, use PostgresStore or other persistent storage)
memory_store = InMemoryStore()

# Pre-populate with sample preferences
# Namespace format: ("category", "identifier")

# Employee 1: Priya - Communication preferences
memory_store.put(
    namespace=("employee_preferences", "EMP101"),
    key="communication_style",
    value={
        "style": "detailed",
        "language": "English",
        "response_format": "paragraphs",
        "last_updated": "2025-01-15"
    }
)

# Employee 1: Priya - Notification settings
memory_store.put(
    namespace=("employee_preferences", "EMP101"),
    key="notification_settings",
    value={
        "email_notifications": True,
        "slack_notifications": True,
        "sms_notifications": False,
        "frequency": "daily"
    }
)

# Employee 1: Priya - Work schedule
memory_store.put(
    namespace=("employee_preferences", "EMP101"),
    key="work_schedule",
    value={
        "preferred_hours": "9AM-5PM",
        "flexible": True,
        "remote_days": ["Monday", "Wednesday", "Friday"],
        "timezone": "Asia/Kolkata"
    }
)

# Employee 2: Rahul - Different preferences
memory_store.put(
    namespace=("employee_preferences", "EMP102"),
    key="communication_style",
    value={
        "style": "concise",
        "language": "English",
        "response_format": "bullet_points"
    }
)

memory_store.put(
    namespace=("employee_preferences", "EMP102"),
    key="notification_settings",
    value={
        "email_notifications": True,
        "slack_notifications": False,
        "sms_notifications": True,
        "frequency": "real-time"
    }
)

# Career goals
memory_store.put(
    namespace=("employee_career", "EMP101"),
    key="goals",
    value={
        "short_term": ["Learn Kubernetes", "Complete AWS certification"],
        "long_term": ["Become Tech Lead", "Mentor junior developers"],
        "last_updated": "2025-01-10"
    }
)

print("✅ Memory store initialized!")
print("\nStored preferences:")
print("  • EMP101: communication_style, notification_settings, work_schedule, career goals")
print("  • EMP102: communication_style, notification_settings")

## Step 2: Create Tools that READ Long-Term Memory

**Pattern:** Use `store: Annotated[InMemoryStore, InjectedStore]` to access memory

In [ ]:
@tool
def get_my_preferences(
    preference_type: Annotated[str, "Type: communication_style, notification_settings, work_schedule"],
    store: Annotated[InMemoryStore, InjectedStore],
    config: RunnableConfig
) -> str:
    """
    Retrieve employee preferences from long-term memory.
    
    This tool demonstrates the get_store() pattern.
    """
    # Get employee ID from runtime config
    employee_id = config.get("configurable", {}).get("employee_id", "UNKNOWN")
    employee = EMPLOYEE_DB.get(employee_id, {})
    name = employee.get("name", "Employee")
    
    # Define namespace for this employee's preferences
    namespace = ("employee_preferences", employee_id)
    
    try:
        # Access long-term memory store
        item = store.get(namespace, preference_type)
        
        if item:
            prefs = item.value
            return f"""✅ **{name}'s {preference_type.replace('_', ' ').title()}:**

```json
{json.dumps(prefs, indent=2)}
```

*Retrieved from long-term memory*"""
        else:
            return f"❌ No {preference_type} found for {name}. Would you like to set them up?"
    
    except Exception as e:
        return f"❌ Error retrieving preferences: {str(e)}"


@tool
def get_all_my_preferences(
    store: Annotated[InMemoryStore, InjectedStore],
    config: RunnableConfig
) -> str:
    """
    Retrieve ALL preferences for the current employee.
    
    Demonstrates searching within a namespace.
    """
    employee_id = config.get("configurable", {}).get("employee_id", "UNKNOWN")
    employee = EMPLOYEE_DB.get(employee_id, {})
    name = employee.get("name", "Employee")
    
    namespace = ("employee_preferences", employee_id)
    
    try:
        # Search for all items in this namespace
        items = store.search(namespace)
        
        if items:
            result = f"**All preferences for {name}:**\n\n"
            
            for item in items:
                result += f"### {item.key.replace('_', ' ').title()}\n"
                result += f"```json\n{json.dumps(item.value, indent=2)}\n```\n\n"
            
            result += f"*Total: {len(items)} preference sets*"
            return result
        else:
            return f"No preferences found for {name}."
    
    except Exception as e:
        return f"Error: {str(e)}"


@tool
def get_career_goals(
    store: Annotated[InMemoryStore, InjectedStore],
    config: RunnableConfig
) -> str:
    """
    Retrieve career development goals from memory.
    
    Uses a different namespace: employee_career
    """
    employee_id = config.get("configurable", {}).get("employee_id", "UNKNOWN")
    employee = EMPLOYEE_DB.get(employee_id, {})
    name = employee.get("name", "Employee")
    
    # Different namespace for career info
    namespace = ("employee_career", employee_id)
    
    try:
        item = store.get(namespace, "goals")
        
        if item:
            goals = item.value
            result = f"**{name}'s Career Goals:**\n\n"
            result += f"**Short-term goals:**\n"
            for goal in goals.get("short_term", []):
                result += f"  • {goal}\n"
            result += f"\n**Long-term goals:**\n"
            for goal in goals.get("long_term", []):
                result += f"  • {goal}\n"
            result += f"\n*Last updated: {goals.get('last_updated', 'Unknown')}*"
            return result
        else:
            return f"No career goals set for {name}."
    
    except Exception as e:
        return f"Error: {str(e)}"

print("✅ READ tools created!")
print("\nTools:")
print("  • get_my_preferences - Get specific preference")
print("  • get_all_my_preferences - Get all preferences")
print("  • get_career_goals - Get career development goals")

## Step 3: Create Tools that WRITE to Long-Term Memory

**Pattern:** Use `store.put()` to save data to memory

In [ ]:
@tool
def update_my_preferences(
    preference_type: Annotated[str, "Type of preference to update"],
    updates: Annotated[str, "JSON string of updates to apply"],
    store: Annotated[InMemoryStore, InjectedStore],
    config: RunnableConfig
) -> str:
    """
    Update employee preferences in long-term memory.
    
    Demonstrates the store.put() pattern for writing memory.
    """
    employee_id = config.get("configurable", {}).get("employee_id", "UNKNOWN")
    employee = EMPLOYEE_DB.get(employee_id, {})
    name = employee.get("name", "Employee")
    
    try:
        # Parse updates
        update_dict = json.loads(updates)
        
        # Get existing preferences
        namespace = ("employee_preferences", employee_id)
        existing = store.get(namespace, preference_type)
        
        if existing:
            # Merge with existing
            current = existing.value.copy()
            current.update(update_dict)
            new_value = current
        else:
            new_value = update_dict
        
        # Add timestamp
        new_value["last_updated"] = datetime.now().isoformat()
        
        # Store updated preferences in long-term memory
        store.put(namespace, preference_type, new_value)
        
        return f"""✅ **Updated {preference_type} for {name}:**

```json
{json.dumps(new_value, indent=2)}
```

*Saved to long-term memory*"""
    
    except json.JSONDecodeError:
        return f"❌ Invalid JSON format. Please provide valid JSON."
    except Exception as e:
        return f"❌ Error updating preferences: {str(e)}"


@tool
def set_career_goal(
    goal: Annotated[str, "Career goal to add"],
    term: Annotated[str, "short_term or long_term"],
    store: Annotated[InMemoryStore, InjectedStore],
    config: RunnableConfig
) -> str:
    """
    Add a career goal to long-term memory.
    
    Demonstrates updating lists in memory.
    """
    employee_id = config.get("configurable", {}).get("employee_id", "UNKNOWN")
    employee = EMPLOYEE_DB.get(employee_id, {})
    name = employee.get("name", "Employee")
    
    if term not in ["short_term", "long_term"]:
        return "❌ Term must be 'short_term' or 'long_term'"
    
    try:
        namespace = ("employee_career", employee_id)
        existing = store.get(namespace, "goals")
        
        if existing:
            goals = existing.value.copy()
        else:
            goals = {"short_term": [], "long_term": []}
        
        # Add new goal
        if goal not in goals[term]:
            goals[term].append(goal)
            goals["last_updated"] = datetime.now().isoformat()
            
            # Save to memory
            store.put(namespace, "goals", goals)
            
            return f"✅ Added {term.replace('_', '-')} goal for {name}: '{goal}'\n\n*Saved to long-term memory*"
        else:
            return f"⚠️  Goal '{goal}' already exists in {term.replace('_', '-')} goals."
    
    except Exception as e:
        return f"❌ Error: {str(e)}"


@tool
def delete_preference(
    preference_type: Annotated[str, "Type of preference to delete"],
    store: Annotated[InMemoryStore, InjectedStore],
    config: RunnableConfig
) -> str:
    """
    Delete a preference from long-term memory.
    
    Demonstrates the store.delete() pattern.
    """
    employee_id = config.get("configurable", {}).get("employee_id", "UNKNOWN")
    employee = EMPLOYEE_DB.get(employee_id, {})
    name = employee.get("name", "Employee")
    
    try:
        namespace = ("employee_preferences", employee_id)
        
        # Check if exists
        existing = store.get(namespace, preference_type)
        
        if existing:
            # Delete from memory
            store.delete(namespace, preference_type)
            return f"✅ Deleted {preference_type} for {name} from long-term memory."
        else:
            return f"⚠️  No {preference_type} found for {name}."
    
    except Exception as e:
        return f"❌ Error: {str(e)}"

print("✅ WRITE tools created!")
print("\nTools:")
print("  • update_my_preferences - Update preferences")
print("  • set_career_goal - Add career goals")
print("  • delete_preference - Remove preferences")

## Step 4: Create Basic Tools

In [ ]:
@tool
def check_leave_balance(
    config: RunnableConfig
) -> str:
    """Check the current employee's leave balance."""
    employee_id = config.get("configurable", {}).get("employee_id", "UNKNOWN")
    employee = EMPLOYEE_DB.get(employee_id, {})
    
    if employee:
        name = employee.get("name")
        balance = employee.get("leave_balance")
        return f"{name} has {balance} days of leave remaining."
    return "Employee not found."

print("✅ Basic tools created!")

---
# Part 3: Combining Dynamic Prompts & Long-Term Memory

Now let's create an agent that uses BOTH:
1. **Dynamic prompts** - Personalized based on memory
2. **Long-term memory tools** - Read and write preferences

## Create the Complete Agent

In [ ]:
# Collect all tools
all_tools = [
    # Read tools
    get_my_preferences,
    get_all_my_preferences,
    get_career_goals,
    # Write tools
    update_my_preferences,
    set_career_goal,
    delete_preference,
    # Basic tools
    check_leave_balance
]

# Create agent with BOTH dynamic prompt AND long-term memory
hr_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=all_tools,
    middleware=[personalized_hr_prompt],  # ✅ Dynamic prompt middleware
    context_schema=HRContext,              # ✅ Context schema
    store=memory_store                     # ✅ Long-term memory store
)

print("✅ Complete HR Agent created!")
print("\nFeatures:")
print("  🎯 Dynamic prompts - Personalized based on user")
print("  📚 Long-term memory - Read preferences")
print("  ✍️ Memory updates - Write preferences")
print("  🔍 Memory search - Find all preferences")
print("  🗂️ Namespaces - Organized storage")

---
# Part 4: Testing the Complete System

## Test 1: Dynamic Prompt with Memory-Based Personalization

In [ ]:
print("=" * 70)
print("TEST 1: Priya (Detailed Style from Memory)")
print("=" * 70)

result = hr_agent.invoke(
    {"messages": [{"role": "user", "content": "Hi! How many leave days do I have?"}]},
    context=HRContext(employee_id="EMP101")  # ✅ Pass context
)

print(f"\n🤖 Response:\n{result['messages'][-1].content}")
print("\n💡 Notice: Response is detailed because Priya's preference (from memory) is 'detailed'")

## Test 2: Different User with Different Preferences

In [ ]:
print("\n" + "=" * 70)
print("TEST 2: Rahul (Concise Style from Memory)")
print("=" * 70)

result = hr_agent.invoke(
    {"messages": [{"role": "user", "content": "Hi! How many leave days do I have?"}]},
    context=HRContext(employee_id="EMP102")  # Different user
)

print(f"\n🤖 Response:\n{result['messages'][-1].content}")
print("\n💡 Notice: Response is concise because Rahul's preference (from memory) is 'concise'")

## Test 3: Read Preferences from Memory

In [ ]:
print("\n" + "=" * 70)
print("TEST 3: Retrieve Preferences from Long-Term Memory")
print("=" * 70)

result = hr_agent.invoke(
    {"messages": [{"role": "user", "content": "What are my notification settings?"}]},
    context=HRContext(employee_id="EMP101")
)

print(f"\n🤖 Response:\n{result['messages'][-1].content}")

## Test 4: Get All Preferences

In [ ]:
print("\n" + "=" * 70)
print("TEST 4: Get All Stored Preferences")
print("=" * 70)

result = hr_agent.invoke(
    {"messages": [{"role": "user", "content": "Show me all my preferences"}]},
    context=HRContext(employee_id="EMP101")
)

print(f"\n🤖 Response:\n{result['messages'][-1].content}")

## Test 5: Update Preferences (Write to Memory)

In [ ]:
print("\n" + "=" * 70)
print("TEST 5: Update Preferences in Long-Term Memory")
print("=" * 70)

result = hr_agent.invoke(
    {"messages": [{"role": "user", "content": 'Turn off my email notifications and set frequency to weekly'}]},
    context=HRContext(employee_id="EMP101")
)

print(f"\n🤖 Response:\n{result['messages'][-1].content}")

## Test 6: Verify Update Persisted

In [ ]:
print("\n" + "=" * 70)
print("TEST 6: Verify Changes Were Saved")
print("=" * 70)

result = hr_agent.invoke(
    {"messages": [{"role": "user", "content": "What are my notification settings now?"}]},
    context=HRContext(employee_id="EMP101")
)

print(f"\n🤖 Response:\n{result['messages'][-1].content}")
print("\n✅ Changes persisted in long-term memory!")

## Test 7: Career Goals

In [ ]:
print("\n" + "=" * 70)
print("TEST 7: Career Goals from Memory")
print("=" * 70)

result = hr_agent.invoke(
    {"messages": [{"role": "user", "content": "What are my career goals?"}]},
    context=HRContext(employee_id="EMP101")
)

print(f"\n🤖 Response:\n{result['messages'][-1].content}")

## Test 8: Add New Career Goal

In [ ]:
print("\n" + "=" * 70)
print("TEST 8: Add New Career Goal")
print("=" * 70)

result = hr_agent.invoke(
    {"messages": [{"role": "user", "content": "Add a short-term goal: Complete Python advanced certification"}]},
    context=HRContext(employee_id="EMP101")
)

print(f"\n🤖 Response:\n{result['messages'][-1].content}")

# Verify it was added
print("\n" + "-" * 70)
print("Verification:")
result = hr_agent.invoke(
    {"messages": [{"role": "user", "content": "What are my career goals now?"}]},
    context=HRContext(employee_id="EMP101")
)

print(f"\n🤖 Response:\n{result['messages'][-1].content}")

---
# Summary

## What We Learned

### 1. Dynamic Prompts with @dynamic_prompt
```python
@dynamic_prompt
def my_prompt(request: ModelRequest) -> str:
    # Access runtime context
    user_id = request.runtime.context.user_id
    
    # Access long-term memory
    store = request.runtime.store
    prefs = store.get(("preferences", user_id), "style")
    
    # Access session state
    msg_count = len(request.state["messages"])
    
    return f"Personalized prompt for {user_id}"

agent = create_agent(
    middleware=[my_prompt],
    context_schema=MyContext,
    store=memory_store
)
```

### 2. Long-Term Memory - Read Pattern
```python
@tool
def read_memory(
    store: Annotated[InMemoryStore, InjectedStore],
    config: RunnableConfig
) -> str:
    user_id = config.get("configurable", {}).get("user_id")
    namespace = ("category", user_id)
    
    # Read from memory
    item = store.get(namespace, "key")
    return item.value if item else "Not found"
```

### 3. Long-Term Memory - Write Pattern
```python
@tool
def write_memory(
    store: Annotated[InMemoryStore, InjectedStore],
    config: RunnableConfig
) -> str:
    user_id = config.get("configurable", {}).get("user_id")
    namespace = ("category", user_id)
    
    # Write to memory
    store.put(namespace, "key", {"data": "value"})
    return "Saved!"
```

## Key Concepts

| Concept | Implementation | Purpose |
|---------|----------------|----------|
| **Dynamic Prompt** | `@dynamic_prompt` decorator | Personalize based on context |
| **Runtime Context** | `request.runtime.context` | Access user-specific info |
| **Long-term Memory** | `InjectedStore` | Persistent storage |
| **Namespaces** | `("category", "id")` | Organize memory |
| **Read Memory** | `store.get()` | Retrieve preferences |
| **Write Memory** | `store.put()` | Save preferences |
| **Search Memory** | `store.search()` | Find all items |
| **Delete Memory** | `store.delete()` | Remove items |

## Production Checklist

- [x] Dynamic prompts personalize experience
- [x] Long-term memory stores preferences
- [x] Tools can read from memory
- [x] Tools can write to memory
- [x] Namespaces organize data
- [ ] Use persistent storage (PostgreSQL, etc.)
- [ ] Add error handling
- [ ] Implement data validation
- [ ] Add audit logging
- [ ] Set up backup strategy

## Next Steps

1. **Switch to persistent storage:**
   ```python
   from langgraph.store.postgres import PostgresStore
   store = PostgresStore(connection_string)
   ```

2. **Add more sophisticated memory:**
   - Semantic search
   - Time-based retrieval
   - Memory summarization

3. **Implement privacy controls:**
   - Data encryption
   - Access permissions
   - GDPR compliance

---

**Congratulations!** You now know how to build context-aware HR agents with dynamic prompts and long-term memory! 🎉